# SpaceX Falcon 9 Interactive Visual Analytics with Folium

## 1. Introduction

This notebook performs interactive visual analytics for SpaceX Falcon 9 launches using Folium.
It builds on previous exploratory analysis (matplotlib/seaborn) and now allows users to interactively explore launch sites, proximity to landmarks, and environmental factors like weather.

Key objectives:

* Visualize all launch sites and their historical launch outcomes.

* Analyze proximities of launch sites to key infrastructure.

* Explore environmental impacts on launch success with interactive maps.

## 2. Library Imports

We import Folium, Pandas, and Folium plugins for interactive features.

In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
import numpy as np
from math import sin, cos, sqrt, atan2, radians

## 3. Load Dataset

We select essential columns for geospatial analysis: launch site name, coordinates, launch success, and weather data.
Grouping by LaunchSite provides unique site locations for mapping.

In [2]:
spacex_df = pd.read_csv("dataset_part_2.csv")
# Keep relevant columns
spacex_df = spacex_df[['LaunchSite', 'Latitude', 'Longitude', 'Class',
                       'TemperatureAvg', 'WindSpeed', 'TempCategory', 'WindCategory', 'PayloadMass']]
launch_sites_df = spacex_df.groupby('LaunchSite', as_index=False).first()[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


## 4. Visualizing Launch Sites on Map

We first plot the four launch sites as circles with labels to give a geographic overview.

In [3]:
# Initialize map
center_coord = [spacex_df['Latitude'].mean(), spacex_df['Longitude'].mean()]
site_map = folium.Map(location=center_coord, zoom_start=5)

# Add circles and labels for each launch site
for launch in launch_sites_df.values:
    coord = [launch[1], launch[2]]
    name = launch[0]
    circle = folium.Circle(coord, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(name))
    marker = folium.map.Marker(coord,
                               icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                                            html=f'<div style="font-size: 12; color:#d35400;"><b>{name}</b></div>'))
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map


## 5. Launch Success for Each Site

We overlay historical launch outcomes as clustered markers.

* Green markers indicate successful launches, red indicates failures.

* Using MarkerCluster reduces clutter from overlapping points at the same launch site.

* Clicking a cluster reveals individual launches, allowing detailed inspection of site performance.

In [4]:
marker_cluster = MarkerCluster()

# Assign marker colors
spacex_df['marker_color'] = np.where(spacex_df['Class']==1, 'green', 'red')

# Add markers to cluster
for _, row in spacex_df.iterrows():
    coord = [row['Latitude'], row['Longitude']]
    marker = folium.Marker(location=coord, icon=folium.Icon(color='white', icon_color=row['marker_color']),
                           popup=f"Launch Site: {row['LaunchSite']}<br>Success: {row['Class']}")
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)
site_map


## 6. Proximities to Launch Site

We analyze a selected launch site (CAFS SLC-40) and its proximity to key landmarks such as railways, highways, coastline, and cities.

### 6.1 Mouse Position for Coordinates

Interactive mouse coordinates allow precise identification of nearby landmarks relative to a launch site, supporting proximity analysis.

In [5]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

## 6.2 Define Landmarks and Calculate Distances

* Lines connect the launch site to landmarks.

* Markers show exact distances, providing clear visual and numerical context.

In [6]:
# Helper function to calculate distance
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Earth radius km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Coordinates of CAFS SLC-40
launch_site_lat = 28.563197
launch_site_lon = -80.576820

coord_dict = {
    "Coast": [28.56235, -80.56781],
    "City": [28.38786, -80.60549],
    "Railway": [28.56162, -80.80204],
    "Highway": [28.52038, -80.6218]
}

for location, coords in coord_dict.items():
    lat, lon = coords
    distance_loc = calculate_distance(launch_site_lat, launch_site_lon, lat, lon)
    distance_marker = folium.Marker([lat, lon],
        icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_loc:.2f} KM to {location}</b></div>'))
    line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [lat, lon]], weight=2, color='blue')
    site_map.add_child(distance_marker)
    site_map.add_child(line)

site_map

### 6.3 Inferences from Proximity Analysis

* Launch sites are near highways for logistical support.

* Launch sites are near coastlines for safe rocket aborts into water.

* Launch sites are away from cities for safety in case of malfunction.

* Launch sites are not close to railways to avoid risk of damage from launches.



## 7. Weather Overlay

This interactive map highlights how temperature affects launch success. Marker color indicates TempCategory (Cold/Moderate/Hot), while popups include payload mass, wind speed, and success.

In [7]:
weather_map = folium.Map(location=center_coord, zoom_start=5)
marker_cluster = MarkerCluster()

def temp_color(temp_cat):
    return {'Cold':'blue','Moderate':'green','Hot':'red'}.get(temp_cat, 'gray')

for _, row in spacex_df.iterrows():
    coord = [row['Latitude'], row['Longitude']]
    popup_text = (f"<b>Launch Site:</b> {row['LaunchSite']}<br>"
                  f"<b>Success:</b> {'Yes' if row['Class']==1 else 'No'}<br>"
                  f"<b>Temp (Avg):</b> {row['TemperatureAvg']}°C<br>"
                  f"<b>Wind (Speed):</b> {row['WindSpeed']} km/h<br>"
                  f"<b>Payload Mass:</b> {row['PayloadMass']} kg")
    marker = folium.Marker(location=coord, popup=popup_text,
                           icon=folium.Icon(color=temp_color(row['TempCategory'])))
    marker_cluster.add_child(marker)

weather_map.add_child(marker_cluster)
weather_map

# Export final folium map to HTML for portfolio use
weather_map.save("folium_weather_launch_sites.html")

## ✅ 8. Conclusion  

This notebook focused on **interactive geospatial analysis** of SpaceX Falcon 9 launch sites using Folium:  

- **Launch site visualization:** Plotted all major launch sites and clustered historical launch outcomes (success vs. failure).  
- **Proximity analysis:** Mapped and measured distances from launch sites to nearby features such as coastlines, highways, railways, and cities, with inferences on why these proximities matter for safety and logistics.  
- **Weather integration:** Added temperature and wind information to maps, providing environmental context for launch site analysis.  

✅ **Skills demonstrated:** interactive geospatial analysis with Folium, marker clustering, distance calculations, layered mapping, and data-driven geographic inference.  

This notebook builds on the earlier EDA notebooks and provides an **interactive, map-based perspective** on Falcon 9 launch sites and their surrounding environments.